The following code draws from the Ray docs at:
https://docs.ray.io/en/releases-1.11.0/ray-core/using-ray-with-pytorch.html

In [62]:
from filelock import FileLock
import argparse
from torchvision import datasets, transforms
import torch
import torch.optim as optim
import ray
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import numpy as np

In [63]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [64]:

def train(model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

In [65]:
def dataset_creator(use_cuda, data_dir):
    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}
    with FileLock("./data.lock"):
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST(
                data_dir,
                train=True,
                download=True,
                transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                ])),
            batch_size=128,
            shuffle=True,
            **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            data_dir,
            train=False,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])),
        batch_size=128,
        shuffle=True,
        **kwargs)

    return train_loader, test_loader

In [66]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # sum up batch loss
            test_loss += F.nll_loss(
                output, target, reduction="sum").item()
            pred = output.argmax(
                dim=1,
                keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    return {
        "loss": test_loss,
        "accuracy": 100 * correct / len(test_loader.dataset)
    }


In [67]:
%run /Users/lucastucker/REU-2023/LE_Experiment_functions.ipynb

In [68]:
def convert_to_np(loader):
    data_list = []
    target_list = []
    for data, target in loader:
        data = data.view(data.size(0), -1) # Flatten into 784 representation
        data_list.append(data.numpy())
        target_list.append(target.numpy())
    # data_array = np.concatenate(data_list)
    # target_array = np.concatenate(target_list)
    return data_array, target_array

In [69]:
def LE_on_loader(loader, t = 5, max_rad = 10, sigma = 0.2, proj_dim = 30):        
    for data, target in loader:
        # print(f"The size of the data is {data.size()}")
        X = data.view(data.size(0), -1).numpy().T # numpy representation 784 x 128
        n, m = X.shape
        N = t_nearest_matr(m, max_rad, X)
        avgs = avg_distance_matr(m, max_rad, X, N)
        W = variable_nbrs_weight(m, max_rad, X, N, avgs)
        P = get_le_reduced(proj_dim, X, W)
        # print(f"The shape of P is {P.shape}")
        # print(f"new data shape is {P.shape}")
        data = torch.from_numpy(P).float()
    return loader

In [70]:
class Network(object):
    def __init__(self, lr=0.01, momentum=0.4, data_dir="~/data"):
        use_cuda = torch.cuda.is_available()
        self.device = device = torch.device("cuda" if use_cuda else "cpu")
        self.train_loader, self.test_loader = dataset_creator(use_cuda, data_dir)
        
        self.train_loader = LE_on_loader(self.train_loader)
        self.test_loader = LE_on_loader(self.test_loader)

        self.model = Model().to(device)
        self.optimizer = optim.SGD(
        self.model.parameters(), lr=lr, momentum=momentum)

    def train(self):
        train(self.model, self.device, self.train_loader, self.optimizer)
        return test(self.model, self.device, self.test_loader)

    def get_weights(self):
        return self.model.state_dict()

    def set_weights(self, weights):
        self.model.load_state_dict(weights)

    def save(self):
        torch.save(self.model.state_dict(), "mnist_cnn.pt")


parser = argparse.ArgumentParser()
parser.add_argument(
    "--data-dir",
    type=str,
    default="~/data/",
    help="Set the path of the dataset."
)
args, unknown = parser.parse_known_args()

net = Network(data_dir=args.data_dir)
net.train()

{'loss': 0.1946672049999237, 'accuracy': 93.96}

In [71]:
RemoteNetwork = ray.remote(num_gpus=2)(Network) # convert Network class to Ray Actor class
NetworkActor = RemoteNetwork.remote() # Instantiate
NetworkActor2 = RemoteNetwork.remote()

ray.get([NetworkActor.train.remote(), NetworkActor2.train.remote()]) # Fetch Ray object reference (future)

2023-09-17 11:58:35,604	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(Network pid=28148) /var/folders/33/zckmdctn235gr36g69vrq8sc0000gn/T/ipykernel_16507/99245488.py:12: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:250.)


[{'loss': 0.15313294439315797, 'accuracy': 95.4},
 {'loss': 0.19187086760997774, 'accuracy': 94.03}]